Prepare the data
- Remove the columns with spurious data:
- Remove the rows with only zeros
- Rename the genes to be sure there is a map with Entrez Gene Identifers
- Rename ages
- Extract the metadata
- Export the counts and metadata

*Generated from a Jupyter Notebook - [Sources](https://github.com/bebatut/neuromac_GF_mices/blob/master/src/prepare_data.ipynb)*

# Load

In [1]:
source("load_libraries.R")

Warning message in is.na(x[[i]]):
“is.na() applied to non-(list or vector) of type 'environment'”Warning message in rsqlite_fetch(res@ptr, n = n):
“Don't need to call dbFetch() for statements, only for queries”

*
*  Package WGCNA 1.63 loaded.
*
*    Important note: It appears that your system supports multi-threading,
*    but it is not enabled within WGCNA in R. 
*    To allow multi-threading within WGCNA with all available cores, use 
*
*          allowWGCNAThreads()
*
*    within R. Use disableWGCNAThreads() to disable threading if necessary.
*    Alternatively, set the following environment variable on your system:
*
*          ALLOW_WGCNA_THREADS=<number_of_processors>
*
*    for example 
*
*          ALLOW_WGCNA_THREADS=4
*
*    To set the environment variable in linux bash shell, type 
*
*           export ALLOW_WGCNA_THREADS=4
*
*     before running R. Other operating systems or shells will
*     have a similar command to achieve the same aim.
*


Allowing multi-threading with up to 4 threads.
[1] "preparing gene to GO mapping data..."
[1] "preparing IC data..."
[1] "preparing gene to GO mapping data..."
[1] "preparing IC data..."
[1] "preparing gene to GO mapping data..."
[1] "prepar

In [2]:
source("functions.R")

In [ ]:
dp = "../../results/dge/"
dir.create(dir_path, showWarnings = FALSE)

Load the gene length

In [3]:
gene_length = read.table("../data/gene_length.tabular", h = F, row.names = 1)
save(gene_length, file=paste(dp, "gene_length.RData", sep=''))

Load the count table

In [4]:
counts = read.table("../data/counts.tabular", sep="\t", header = 1, row.names=1)
dim(counts)

[1] 24421    64

# Remove spurious data

Remove the columns with spurious data:
- SPF_8w_F_2_2: low mapping rate (70.9%) and assignment rate (31.3%)
- SPF_8w_F_5_2: low assignment rate (53.5%)
- SPF_8w_M_1_2: low number of assigned reads (14.4)

In [5]:
to_remove = c("SPF_8w_F_2_2", "SPF_8w_F_5_2", "SPF_8w_M_1_2")
counts = counts[,!names(counts) %in% to_remove]

Remove the rows with only zeros (printed: before / after)

In [6]:
nrow(counts)
counts = counts[ rowSums(counts) > 1, ]
nrow(counts)

[1] 24421

[1] 19694

# Check and clean genes names

Rename the genes to be sure there is a map with Entrez Gene Identifers:
1. Extract mapping between 
    - Entrez Gene Identifiers and Gene Names (and description) 
    - Gene Symbols and Entrez Gene Identifiers
    - RefSeq and Entrez Gene Identifiers
    - Gene Symbols and Gene Names (and description)
    - Entrez Gene Identifiers and KEGG pathways
2. Get RefSeq ids for genes in count table
3. Get genes without RefSeq ids (printed: number of them that will need to be changed)
4. Search on Entrez the correct name of the genes without RefSeq ids
5. Clean the new names to keep only the ones with one new names, that are not already in the count table,
6. Apply them to the counts
7. Check the new number of genes without RefSeq ids

In [7]:
# Extract mapping between Entrez Gene Identifiers and Gene Names (and description) 
eg2name = get_list(org.Mm.egGENENAME)
# Extract mapping between Gene Symbols and Entrez Gene Identifiers
symbol2eg = get_list(org.Mm.egSYMBOL2EG)
# Extract mapping between RefSeq and Entrez Gene Identifiers
refseq2eg = get_list(org.Mm.egREFSEQ2EG)
# Extract mapping between Gene Symbols and Gene Names (and description)
symbol2name = sapply(names(symbol2eg), function(x) return(eg2name[symbol2eg[[x]]]))
# Extract mapping between Entrez Gene Identifiers and KEGG pathways
eg2kegg=as.list(org.Mm.egPATH)
# Get refseq ids for genes
refseq = sapply(rownames(counts), function(x) return(symbol2eg[[x]])) 
# Extract names of genes with no RefSeq ids
to_change = names(refseq[sapply(refseq, is.null)])
length(to_change)
# Search on Entrez the correct name
search_name = function(name){
    search = entrez_search(db="gene",term=name)
    names = c()
    for(id in search$ids){
        sum = entrez_summary(db="gene", id=id)
        if(sum$organism$scientificname == 'Mus musculus' & grepl(name,sum$otheraliases)){
            names = c(names, sum$name)
        }
    }
    return(names)
}
changes = sapply(to_change, search_name)
# Clean the new names
doubled = sapply(names(changes), function(x) return(length(changes[[x]])>1))
single = changes[!doubled]
duplicated = single %in% rownames(counts)
non_duplicated = single[!duplicated]
duplicated_2 = duplicated(non_duplicated)
replacements = non_duplicated[!duplicated_2]
# Replace the names
replacements = unlist(replacements)
new_names = rownames(counts)
names(new_names) = rownames(counts)
new_names[names(replacements)] = replacements
head(new_names)
rownames(counts) = new_names
# Check the new number of genes without RefSeq ids
refseq_check = sapply(rownames(counts), function(x) return(symbol2eg[[x]])) 
length(refseq[sapply(refseq_check, is.null)])

[1] 231

0610005C13Rik   0610007P14Rik   0610009B22Rik   0610009L18Rik   0610009O20Rik 
"0610005C13Rik" "0610007P14Rik" "0610009B22Rik" "0610009L18Rik" "0610009O20Rik" 
  0610010F05Rik 
"0610010F05Rik"

[1] 29

# Order the data

Reorder the columns and save the counts object

In [8]:
head(counts)
counts = counts %>%
         rownames_to_column('gene_names') %>%
         select(sort(current_vars())) %>%
         column_to_rownames('gene_names')
head(counts)
save(counts, file=paste(dp, "prepared_counts.RData", sep=''))

,GF_104w_F_1_2,GF_104w_F_2_2,GF_104w_F_3_2,GF_104w_M_1_2,GF_104w_M_2_2,GF_52w_M_1_2,GF_52w_M_2_2,GF_52w_M_3_2,GF_52w_M_4_2,GF_8w_M_1_2,⋯,SPF_8w_F_3_2,SPF_8w_F_4_2,GF_8w_F_1_2,GF_8w_F_2_2,GF_8w_F_3_2,GF_8w_F_4_2,GF_8w_F_5_2,GF_104w_M_3_2,GF_104w_M_5_2,GF_104w_M_4_2
0610005C13Rik,0,0,0,2,2,1,5,5,10,7,⋯,2,2,17,0,2,5,0,1,1,1
0610007P14Rik,954,704,999,1002,1577,604,537,459,1021,499,⋯,705,698,837,627,605,563,631,394,570,496
0610009B22Rik,1122,1098,1296,1385,2475,1234,887,907,1214,784,⋯,1103,996,969,938,1128,911,853,551,772,522
0610009L18Rik,31,34,20,23,61,38,27,20,6,51,⋯,43,79,45,18,11,43,19,20,22,26
0610009O20Rik,239,262,292,256,432,356,354,353,428,298,⋯,239,175,216,146,268,289,210,269,351,263
0610010F05Rik,452,433,317,315,483,466,411,347,495,284,⋯,365,228,505,368,402,502,251,412,540,207


,GF_104w_F_1_2,GF_104w_F_2_2,GF_104w_F_3_2,GF_104w_M_1_2,GF_104w_M_2_2,GF_104w_M_3_2,GF_104w_M_4_2,GF_104w_M_5_2,GF_52w_F_1_2,GF_52w_F_2_2,⋯,SPF_52w_M_2_2,SPF_52w_M_3_2,SPF_52w_M_4_2,SPF_52w_M_5_2,SPF_8w_F_1_2,SPF_8w_F_3_2,SPF_8w_F_4_2,SPF_8w_M_2_2,SPF_8w_M_3_2,SPF_8w_M_4_2
0610005C13Rik,0,0,0,2,2,1,1,1,5,2,⋯,9,0,3,9,3,2,2,9,8,4
0610007P14Rik,954,704,999,1002,1577,394,496,570,697,701,⋯,919,604,767,760,654,705,698,615,648,745
0610009B22Rik,1122,1098,1296,1385,2475,551,522,772,1092,1257,⋯,1485,1102,1252,1137,1087,1103,996,985,941,1063
0610009L18Rik,31,34,20,23,61,20,26,22,27,38,⋯,84,47,53,60,36,43,79,27,55,76
0610009O20Rik,239,262,292,256,432,269,263,351,309,330,⋯,646,422,428,352,235,239,175,493,348,429
0610010F05Rik,452,433,317,315,483,412,207,540,338,373,⋯,531,361,467,466,292,365,228,313,303,355


Load the gene length, keep the ones in count table and rename the gene names 

In [9]:
gene_length = read.table("../data/gene_length.tabular", h = F, row.names = 1)
gene_length = gene_length[rownames(counts),]
names(gene_length) = new_names
save(gene_length, file=paste(dp, "gene_length.RData", sep=''))

# Extract metadata

Extract the description of the samples

In [10]:
file_desc = read.csv("../data/file_description.csv", row.names = 1) %>%
            rownames_to_column('sample') %>%
            slice(1:(n()-6)) %>%
            mutate(Lane = gsub(" & ", "_", Lane)) %>%
            mutate(Lane = gsub(" ", "1", Lane)) %>%
            mutate(Project.id = gsub("Project_", "", Project.id)) %>%
            mutate(Project.id = gsub("148", "S148", Project.id))
dim(file_desc)
head(file_desc)

[1] 64 10

sample,Sample.name.prefix,Group,Age,Gender,Project.id,Lane,Replicate,Name.in.project,X..Reads
GF_52w_M_1,GF_52w_M,GF,52w,M,S148,2,1,Sample_Mg_GF1_old,"35,713,942.00"
GF_8w_M_1,GF_8w_M,GF,8w,M,S148,2,1,Sample_Mg_GF1_young,"25,919,398.00"
GF_52w_M_2,GF_52w_M,GF,52w,M,S148,2,2,Sample_Mg_GF2_old,"29,752,263.00"
GF_8w_M_2,GF_8w_M,GF,8w,M,S148,2,2,Sample_Mg_GF2_young,"24,133,081.00"
GF_52w_M_3,GF_52w_M,GF,52w,M,S148,3,3,Sample_Mg_GF3_old,"26,395,568.00"
GF_8w_M_3,GF_8w_M,GF,8w,M,S148,1,3,Sample_Mg_GF3_young,"33,992,780.00"


Extract the metadata:
1. Extract from the counts
2. Combine with the sample descriptions
3. Rename the ages
  - 8w to Young
  - 52w to Middle-aged
  - 104w to Old
4. Rename the gender
  - F to Female
  - M to Male
5. Rename the factors
  - gender to Sex
  - type to Microbiota
  - age to Age

In [11]:
metadata = tbl_df(t(sapply(sapply(colnames(counts), strsplit, split = "_"), unlist))) %>%
   magrittr::set_colnames(c("type", "age", "gender", "replicate", "extra")) %>%
   mutate(sample = paste(type, age, gender, replicate, extra, sep ="_")) %>%
   select(-extra) %>%
   mutate(short_name = paste(type, age, gender, replicate, sep ="_")) %>%
   select(-replicate) %>%
   arrange(short_name) 
projects = file_desc %>%
   filter(sample %in% metadata$short_name) %>%
   arrange(sample) %>%
   pull("Project.id")
metadata = metadata %>%
    mutate(project = projects) %>%
    select(-short_name) %>%
    mutate(age = gsub("104w", "Old", age )) %>%
    mutate(age = gsub("52w", "Middle-aged", age )) %>%
    mutate(age = gsub("8w", "Young", age )) %>%
    mutate(gender = gsub("F", "Female", gender )) %>%
    mutate(gender = gsub("M", "Male", gender)) %>%
    rename(Sex = gender) %>%
    rename(Microbiota = type) %>%
    rename(Age = age)
metadata %>%
    group_by(Microbiota, Age, Sex)  %>% 
    summarise(Number = n())
metadata

Microbiota,Age,Sex,Number
GF,Middle-aged,Female,6
GF,Middle-aged,Male,4
GF,Old,Female,3
GF,Old,Male,5
GF,Young,Female,5
GF,Young,Male,4
SPF,Middle-aged,Female,6
SPF,Middle-aged,Male,5
SPF,Old,Female,3
SPF,Old,Male,14


Microbiota,Age,Sex,sample,project
GF,Old,Female,GF_104w_F_1_2,S264
GF,Old,Female,GF_104w_F_2_2,S264
GF,Old,Female,GF_104w_F_3_2,S264
GF,Old,Male,GF_104w_M_1_2,S264
GF,Old,Male,GF_104w_M_2_2,S264
GF,Old,Male,GF_104w_M_3_2,S288
GF,Old,Male,GF_104w_M_4_2,S288
GF,Old,Male,GF_104w_M_5_2,S288
GF,Middle-aged,Female,GF_52w_F_1_2,S178
GF,Middle-aged,Female,GF_52w_F_2_2,S178


In [12]:
save(metadata, file=paste(dp, "metadata.RData", sep=''))

# Citations

In [13]:
citation("rentrez")


To cite rentrez in publications use:

  Winter, D. J. (2017) rentrez: an R package for the NCBI eUtils API
  The R Journal 9(2):520-526

A BibTeX entry for LaTeX users is

  @Article{,
    title = {{rentrez}: an R package for the NCBI eUtils API},
    author = {David J. Winter},
    journal = {The R Journal},
    year = {2017},
    volume = {9},
    issue = {2},
    pages = {520--526},
  }
